The (Classical) Hough Transform
===============================

In [1]:
import bqplot.pyplot as plt
import bqplot as bq
import numpy as np
import ipywidgets as widgets

The _classical_ Hough transform is a feature extraction technique used to find a subset of geometrical shapes within an image subject to imperfections. 

Lines
-----

In polar space, a line may be parametrised in *Hesse normal form* by the length $r$ of the normal to the origin $\vec{r}$, and the angle $\theta$ of $\vec{r}$ with respect to the origin:
$$
r(\theta) = x_i\cos(\theta) + y_i\sin(\theta)\,.
$$
For any point $(x, y)$ on this line, $r$ and $\theta$ are *constant*.

In [203]:
x = np.linspace(-200, 200)
    
plt.figure(max_aspect_ratio=1, min_aspect_ratio=1)
line = plt.plot(x=x, y=x*0, labels=['Line'], 
                options={'y': {'min': -x.max(), 'max': x.max()}, 
                         'x': {'min': -x.max(), 'max': x.max()}})
normal = plt.plot(x, x*0, 'r--', labels=['Normal'])
label = plt.label(text=[],x=[],y=[], colors=['orange'])
plt.legend()
plt.show()

y0 = -200
x0 = -200

@widgets.interact
def draw(m=widgets.FloatLogSlider(min=-2, max=2, description="-m"), c=(-400.0, 400.0)):
    m = -m
    y = m*x + c
    line.y = y
    normal.y = (-1/m)*(x-x0) + y0  
    z = np.pi/2 - np.arctan(-m)
    r = 200
    label.y = [r*np.sin(z)+y0]
    label.x = [r*np.cos(z)+x0]
    label.text = [f"({c/(np.sin(z)-m*np.cos(z)):.3f}, {np.degrees(z):.3f})"]

interactive(children=(FloatLogSlider(value=1.0, description='-m', max=2.0, min=-2.0), FloatSlider(value=0.0, d…

In [195]:
plt.figure()
t = 0
s = np.radians(75.9)
plt.plot(x, (t - x*np.cos(s))/np.sin(s), 
         options={'y': {'min': -x.max(), 'max': x.max()}, 
                  'x': {'min': -x.max(), 'max': x.max()}})
plt.show()

## Define two lines

In [210]:
x = np.linspace(0, 100, 100)

y1 = 3*x + 30
y2 = 1.5*x + 40

In [211]:
plt.figure() 
plt.plot(x, y1)
plt.plot(x, y2)
plt.show()


In the context of image analysis, the cartesian coordinates $(x_i,y_i)$ are known, whilst $(r,\theta)$ are the parameters to be solved for. Given that for all points on a straight line, $r$ and $\theta$ remain constant, it follows that there exist many lines upon which $(x_i, y_i)$ lies, and thus each $(x_i, y_i)$ in cartesian space maps to _curves_ in the polar Hough parameter space. Hence, points which are colinear in cartesian space (i.e. lie on the same line) will map to curves in Hough space which intersect at some $(r, \theta)$.


The transform is implemented by transforming each $(x_i, y_i)$ into Hough parameter space, and then incrementing an accumulator each $(r_i, \theta_i)$ along a discretisation of the $(r,\theta)$ curve. Identifying peaks in the resulting $(r,\theta)$ histogram indicates the presence of a line with the given paramete

### Build $\theta$ and $r$ arrays from known $(x_i,y_i)$ points

In [212]:
theta1 = np.tile(np.linspace(0, 2*np.pi, 200).reshape(-1,1), len(x))
r1 = x*np.cos(theta1) + y1*np.sin(theta1)

### Plot in a histogram

In [216]:
hist_2d, r1_edges, theta1_edges = np.histogram2d(r1.ravel(),  theta1.ravel(), bins=100)

In [217]:
plt.figure()
plt.heatmap(hist_2d, x=r1_edges, y=np.degrees(theta1_edges), cmap="viridis", tooltip=bq.Tooltip(fields=['midpoint']))
plt.xlabel("r")
plt.ylabel("theta")
plt.show()

### Extract the top $4$ indices by frequency

In [220]:
max_indices = np.vstack(np.unravel_index(np.argsort(hist_2d, axis=None), hist_2d.shape)).T
maxima = max_indices[-1:-5:-1]

### Plot fit parameters

In [221]:
i_r1, j_theta1 = maxima[0]
r1_fit = r_edges[i_r1]
theta1_fit = theta_edges[j_theta1]

In [209]:
plt.figure()
plt.plot(x, (r1_fit - x*np.cos(theta1_fit))/np.sin(theta1_fit), "r--", labels=['Fit'])
plt.plot(x, y1, labels=['True'])
plt.legend()
plt.show()